- packages

In [1]:
# general
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import os, yaml, requests, re, string, random
import numpy as np

- functions

In [2]:
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """

    config = None
    try:
        with open(yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')

    return config


def get_db_conn_post(creds_file):
    """ Get an authenticated psycopg db connection, given a credentials file"""
    creds = read_yaml_file(creds_file)['db']

    connection = psycopg2.connect(
        user=creds['user'],
        password=creds['pass'],
        host=creds['host'],
        port=creds['port'],
        database=creds['db']
    )

    return connection

- set up the connection to the database

In [3]:
db_conn_post = get_db_conn_post("key_postgresql.yaml")

- **Write the SQL queries necessary to generate a list of the five restaurants that have the highest average number of visitors on holidays. The result table should also contain that average per restaurant.**

In [4]:
q = """
    select 
            a.id as restaurant, 
            avg(a.reserve_visitors) as avg_visitors
    from
            kaggle_data.didi.restaurants_visitors as a
    left join 
            kaggle_data.didi.date_info as b
    on
            a.visit_date = b.calendar_date
    where 
            b.holiday_flg = 1
    group by  
            a.id
    order by
            avg_visitors desc
    limit 
            5;        
"""

In [5]:
pd.read_sql(sql = q, con = db_conn_post)

C:\Users\migue\AppData\Local\Temp/ipykernel_6716/2549109559.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql = q, con = db_conn_post)


,restaurant,avg_visitors
0,e89735e80d614a7e,8.500000
1,db80363d35f10926,6.904110
2,bcce1ea4350b7b72,5.500000
3,e053c561f32acc28,5.184615
4,bb09595bab7d5cfb,5.142857


- **Use SQL to discover which day of the week there are usually more visitors on average in restaurants.**

In [6]:
q = """
    select
            round(avg(a.reserve_visitors), 2) as avg_visitors, b.day_of_week 
    from
            kaggle_data.didi.restaurants_visitors as a
    left join
            kaggle_data.didi.date_info as b
    on
            a.visit_date = b.calendar_date
    group by 
            b.day_of_week 
    order by 
            avg_visitors desc;
"""

In [7]:
pd.read_sql(sql = q, con = db_conn_post)

C:\Users\migue\AppData\Local\Temp/ipykernel_6716/2549109559.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql = q, con = db_conn_post)


,avg_visitors,day_of_week
0,4.31,Friday
1,4.19,Wednesday
2,4.19,Thursday
3,4.11,Tuesday
4,4.03,Monday
5,3.94,Saturday
6,3.41,Sunday


now we try another method (pandas)

In [8]:
q = 'select * from kaggle_data.didi.restaurants_visitors;'

In [9]:
df = pd.read_sql(sql = q, con = db_conn_post)

C:\Users\migue\AppData\Local\Temp/ipykernel_6716/356735535.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql = q, con = db_conn_post)


In [10]:
df.head(1)

,id,visit_date,visit_datetime,reserve_datetime,reserve_visitors
0,db80363d35f10926,2016-01-01,2016-01-01,2016-01-01 01:00:00,5


In [11]:
df['day_of_week'] = df['visit_date'].dt.day_name()

In [12]:
df.head(1)

,id,visit_date,visit_datetime,reserve_datetime,reserve_visitors,day_of_week
0,db80363d35f10926,2016-01-01,2016-01-01,2016-01-01 01:00:00,5,Friday


now with the name of the day, we can obtain the data without missing values

In [13]:
columns = ['reserve_visitors', 'day_of_week']
df[columns]\
.groupby(by = 'day_of_week', as_index = False)\
.agg('mean')\
.sort_values(by = 'reserve_visitors', ascending = False)

,day_of_week,reserve_visitors
0,Friday,4.311475
6,Wednesday,4.190618
4,Thursday,4.185874
5,Tuesday,4.108614
1,Monday,4.032590
2,Saturday,3.939041
3,Sunday,3.410092


- **How was the percentage of growth of the amount of visitors week over week for the last four weeks of the data?**

In [14]:
q = """
with visitors as(
    select 
            extract(year from visit_date) as year_data,
            extract(month from visit_date) as month_data,
            extract(week from visit_date) as week_data,
            sum(reserve_visitors) as cu_data
    from 
            kaggle_data.didi.restaurants_visitors as rv
    group by 
            year_data, month_data, week_data
    )
    select 
            year_data as year, 
            month_data as month, 
            week_data + 1 as week_of_year,
            round((lag(cu_data) over (order by year_data desc, month_data desc, week_data desc) - cu_data) / cu_data, 3) * 100 as pct_variation
    from 
            visitors
    order by 
            year_data desc, month_data desc, week_data desc;
"""

In [15]:
pd.read_sql(sql = q, con = db_conn_post)

C:\Users\migue\AppData\Local\Temp/ipykernel_6716/2549109559.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql = q, con = db_conn_post)


,year,month,week_of_year,pct_variation
0,2017.0,5.0,23.0,NaN
1,2017.0,5.0,22.0,-82.5
2,2017.0,5.0,21.0,-19.2
3,2017.0,5.0,20.0,-54.1
4,2017.0,5.0,19.0,30.8
...,...,...,...,...
70,2016.0,1.0,54.0,280.4
71,2016.0,1.0,5.0,-72.5
72,2016.0,1.0,4.0,-23.3
73,2016.0,1.0,3.0,15.7
